In [1]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from Seattle_airbnb import Seattle_airbnb
import datetime as dt
from config import api_key

importing Jupyter notebook from Seattle_airbnb.ipynb


Seattle_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
Seattle_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
Seattle_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [2]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Seattle, WA")

WA_coord = []
WA_coord.append(location.latitude)
WA_coord.append(location.longitude)

WA_coord

[47.6038321, -122.3300624]

In [3]:
forecast = forecastio.load_forecast(api_key, WA_coord[0], WA_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Light rain today through Monday, with high temperatures bottoming out at 45°F on Tuesday.
rain
45.72


In [4]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [5]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [6]:
Seattle_airbnb.dtypes

Date         object
Under $50     int64
$50-100       int64
$100-150      int64
$150-200      int64
$200-250      int64
dtype: object

In [7]:
Seattle_table = pd.merge(df, Seattle_airbnb, on="Date")
Seattle_table["City"] = "Seattle"
Seattle_table


,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250,City
0,2018-12-22,47.67,44.85,0.84,0.0136,rain,357,1268,1051,571,277,Seattle
1,2018-12-23,47.36,38.78,0.94,0.0102,rain,375,1166,982,531,285,Seattle
2,2018-12-24,45.89,35.84,0.83,0.0069,rain,355,1119,977,545,282,Seattle
3,2018-12-25,44.67,36.63,0.43,0.0016,rain,342,1095,958,532,286,Seattle
4,2018-12-26,45.72,37.93,0.88,0.0073,rain,353,1145,981,540,310,Seattle
5,2018-12-27,45.25,35.49,0.53,0.0019,rain,362,1148,930,565,327,Seattle
6,2018-12-28,45.77,39.63,0.72,0.0047,rain,330,1057,946,543,386,Seattle
7,2018-12-29,54.08,46.50,0.96,0.0153,rain,319,1070,938,545,396,Seattle
